<a href="https://colab.research.google.com/github/lcbjrrr/quant/blob/master/IA_R_Clas_RegLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall rpy2 -y

Found existing installation: rpy2 3.5.5
Uninstalling rpy2-3.5.5:
  Successfully uninstalled rpy2-3.5.5


In [ ]:
!pip install rpy2==3.0.0

In [4]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [14]:
%%R
#install.packages("glm",repos = "http://cran.us.r-project.org")

NULL


In [ ]:
%%R
#library("glm2")

In [38]:
%%R 
Score <- function(mod,Xs,y_exp,t=""){
   if(t=="response"){
     prevs<-predict(mod,Xs,type=t)>0.5
   }else if(t=="knn"){
     df<-data.frame(y_exp,Xs)
     prevs<-mod(df,df,cl=df$y,k=3)
   }else{
     prevs<-predict(mod,Xs)   
   }
   score<-sum(prevs==y_exp)/length(prevs)
   return(score)
}
 
Accuracy <- function(y_exp,prevs){
   accuracy<-sum(prevs==y_exp)/length(prevs)
   return(accuracy)
}
 
ConfusionMatrix <- function(y_exp,prevs){ 
   cm<-table(y_exp,prevs)
   return(cm)
} 

PrecisionRecall <- function(y_exp,prevs){
   cm<-table(y_exp,prevs)
   tp<-cm[2,2]
   tn<-cm[1,1]
   fn<-cm[2,1]
   fp<-cm[1,2]
   precision <- tp/(tp+fp)
   recall <- tp/(tp+fn)
   return(c(precision,recall)) 
}


In [13]:
%%R
train <- read.csv('https://raw.githubusercontent.com/lcbjrrr/data/main/gender%20-%20tr.csv')
print(train)
rlog <- glm( G ~ . , data = train, family='binomial')
rlog


  G   H  W
1 0 178 72
2 0 179 81
3 1 163 55
4 1 168 58
5 0 181 98
6 1 170 60
7 0 184 78
8 1 171 59

Call:  glm(formula = G ~ ., family = "binomial", data = train)

Coefficients:
(Intercept)            H            W  
  1243.0080      -7.2523       0.3435  

Degrees of Freedom: 7 Total (i.e. Null);  5 Residual
Null Deviance:	    11.09 
Residual Deviance: 3.56e-10 	AIC: 6


In [17]:
%%R
test <- read.csv('https://raw.githubusercontent.com/lcbjrrr/data/main/gender%20-%20ts.csv')
test[,2:3]

    H  W
1 175 75
2 165 65


In [25]:
%%R
pred_test<- factor(predict(rlog,test[,2:3],type = "response")>0.5,levels=c(F,T))
pred_test

    1     2 
FALSE  TRUE 
Levels: FALSE TRUE


In [30]:
%%R 
print("ACC Train: ")
Score(rlog,train[,2:3],train[,1],"response")

[1] "ACC Train: "
[1] 1


In [36]:
%%R
print("ACC Test: ")
y_ref <- factor(test[,1]>0.5,levels=c(F,T))
Accuracy(y_ref,pred_test)

[1] "ACC Test: "
[1] 1


In [39]:
%%R
print("CM Test: ")
ConfusionMatrix(y_ref,pred_test)

[1] "CM Test: "
       prevs
y_exp   FALSE TRUE
  FALSE     1    0
  TRUE      0    1


In [42]:
%%R
print("Precision / Recall - Test: ")
PrecisionRecall(y_ref,pred_test)

[1] "Precision / Recall - Test: "
[1] 1 1
